In [24]:
"""
Usage:
    python nodes.py [NUM_TRIALS] [LEN_N_RANGE]
For example:
    python nodes.py 500 6
represents simulating the number of reservoir nodes at 100, 200, 400, 800, 1600, 
3200. For each simulation, we perform 500 trials to smooth out the randomness.
"""

import sys
import os
from datetime import datetime

import numpy as np

import itertools
import multiprocessing

from reservoir import reservoir as res
from data import data
from error import error

################################################################################
###############################-----CONSTANTS-----##############################
################################################################################
# define constants 
LAMBDA = 0.9056     # lorenz lyapunov exponent
SEED = 42
STATE = np.random.RandomState(SEED)

hyperparams = {
    'GAMMA': 7.7,
    'SIGMA': 0.81,
    'RHO_IN': 0.37,
    'K': 3,
    'RHO_R': 0.41,
    'N': 100
}

simulation_parameters = {
    "DEL_T": 0.02,          # time step size
    "STEPS": 30000,         # total steps
    "WASHOUT": 10000,       # washout 
    # n = STEPS - WASHOUT
    "ALPHA": 0.001,         # Tikhonov regularisation constant
    "d": 3,
    "INITIAL_STATE_RANGE": 5
}

# NUM_TRIALS = int(sys.argv[1])
NUM_TRIALS = 10
trial_range = range(NUM_TRIALS)
# NUM_N_VALS = int(sys.argv[2])
NUM_N_VALS = 2
N_range = [100 * (2 ** n) for n in range(NUM_N_VALS)]

training_data_ls = []
test_data_ls = []

################################################################################
############################-----HELPER FUNCTIONS-----##########################
################################################################################
# make a directory
def make_dir(dir):
    try:
        os.mkdir(dir)
        print("Created directory: {}".format(dir))
        return True
    except FileExistsError:
        return False

def generate_initial_states(num_trials, simulation_parameters):
    d = simulation_parameters["d"]
    initial_state_range = simulation_parameters["INITIAL_STATE_RANGE"]
    
    training_initial_states = STATE.uniform(-initial_state_range, initial_state_range, (num_trials, d))
    test_initial_states = STATE.uniform(-initial_state_range, initial_state_range, (num_trials, d))

    return training_initial_states, test_initial_states

training_initial_states, test_initial_states = generate_initial_states(
    num_trials=NUM_TRIALS,
    simulation_parameters=simulation_parameters
)

In [25]:
def generate_datasets(trial_id):
    training_data_ls.append(data.generate_lorenz_63(
        initial_state=training_initial_states[trial_id],
        del_t=simulation_parameters["DEL_T"],
        steps=simulation_parameters["STEPS"],
        washout=simulation_parameters["WASHOUT"]
    ))

    test_data_ls.append(data.generate_lorenz_63(
        initial_state=test_initial_states[trial_id],
        del_t=simulation_parameters["DEL_T"],
        steps=simulation_parameters["STEPS"],
        washout=simulation_parameters["WASHOUT"]
    ))
    print("\tDataset #{} generated.".format(trial_id))

    return training_data_ls, test_data_ls

In [26]:
for i in trial_range:

    generate_datasets(i)

	Dataset #0 generated.
	Dataset #1 generated.
	Dataset #2 generated.
	Dataset #3 generated.
	Dataset #4 generated.
	Dataset #5 generated.
	Dataset #6 generated.
	Dataset #7 generated.
	Dataset #8 generated.
	Dataset #9 generated.


In [30]:
for set in test_data_ls:
        print(set)
        print("---")

[[ 3.25967062  1.67617682 23.8976943 ]
 [ 2.99589436  1.93124792 22.7657895 ]
 [ 2.82984199  2.22485889 21.70076078]
 ...
 [ 6.79100779 11.18779938 15.89950332]
 [ 7.72353072 12.64486434 16.75935572]
 [ 8.7547189  14.12632419 18.03996424]]
---
[[ -0.86470361  -1.53943059  13.59746488]
 [ -1.00899603  -1.78224781  12.92151574]
 [ -1.17613031  -2.07969644  12.29144153]
 ...
 [ -7.88180149 -15.53921366   8.72237766]
 [ -9.53096725 -18.34603259  11.14631394]
 [-11.39197415 -21.12046677  14.59994051]]
---
[[ 3.95571417 -4.22813148 31.84843287]
 [ 2.45949864 -4.32976117 29.92859827]
 [ 1.23159603 -4.28556914 28.22122992]
 ...
 [ 1.87389761  3.94172038 21.88433017]
 [ 2.26558646  4.13579136 20.91077283]
 [ 2.62984442  4.4201433  20.02876549]]
---
[[ -4.33951109  -6.98798363  15.09445819]
 [ -4.91576241  -8.03692604  14.98717454]
 [ -5.59079548  -9.226005    15.09162475]
 ...
 [ -4.29830037  -7.4251001   12.89762653]
 [ -4.97828763  -8.66177996  12.95394816]
 [ -5.77573563 -10.0753202   13.262